In [ ]:
"""
instance - an XBRL document whose root element is <xbrli:xbrl>
fact - An individual detail in a report, such as $20M
concept - The meaning associated with a fact, such as the cost of goods sold
entity - The company or individual described by a concept
context - A data structure that associates an entity with a concept
"""

# SEC uses Inline XBRL which 
# involves embedding XBRL data directly into the filing so that the
# disclosure document is both human-readable and machine-readable. 

# python-xbrl has issues??
# use lxml instead

In [47]:
from bs4 import BeautifulSoup
import xbrl
from requests import get
import sys
import re
import nltk
import urllib.request

"""
So, from zip file, extract the html of the 10-q report into text
"""

# microsoft = sft-10q_20191231.htm
# apple = a10-qq1202012282019.htm
# facebook = fb-09302019x10q.htm
# netflix = form10q_q119.htm
# dropbox = dbx-093019x10q.htm


def html_to_text(url):
    '''
    :param url: [string] to a sec websites
    :return: document [string], with no processing
    '''
    #Pretty self explanatory, but is easier than remebering this syntax every time
    html = urllib.request.urlopen(url)
    return BeautifulSoup(html,features="html.parser").get_text()

print(html_to_text('https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019320000010/a10-qq1202012282019.htm'))

def extractFromXBRL(xbrl_dir):
    xbrl_str = open(xbrl_dir)
    text = BeautifulSoup(xbrl_str, "lxml").get_text()
    return text

#print(extractFromXBRL('C:/Users/jsbae/financialsummary/resources/xml_files/a10-qq1202012282019.htm'))

def makeText(url, index):
    #Retrieve document
    document = html_to_text(url)

    #An example of retrieving sentences
    sentences = nltk.sent_tokenize(document)
    print(len(sentences))
    num_good_sents = 0
    good_sents = []
    def check_for_formatting(string):
        #Ignore sentences with annoying syntax
        #TODO: figure out how to remove backslash and maybe also remove tabs?
        regex = re.compile('[☒_☐@#^&*()<>?/\|}{~:]|\s\s')
        if(regex.search(string) == None):
            return True

    for sentence in sentences:
        if check_for_formatting(sentence):
            num_good_sents = num_good_sents+1
            good_sents.append(sentence)
            # print(sentence)

    print("Total acceptable sentences =",num_good_sents)
    #Converting the sentences to a document
    good_doc = " ".join(good_sents)
    return good_doc
    
#makeText('https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019320000010/a10-qq1202012282019.htm', 1)











  (function( ) {
    var hostName = window.location.hostname;
    
    var browserAccepted = function( ) {
	
      var ua = window.navigator.userAgent;
      var trident = ua.indexOf('Trident/'); //IE 11;
      var msie = ua.indexOf('MSIE '); // IE 10 or older
      if ( msie > 0 ) {
        return false;
      }
      return true;
    }();
    
    var pathToLibs = './js';
    var pathToError = '/';
    if ( hostName === 'www-test.sec.gov' || hostName === 'www.sec.gov' ) {
      pathToLibs = './ixviewer/js';
      pathToError = '/ixviewer/';
    }
    if ( browserAccepted ) {
      
      // favicon
      <!-- document.write('<link rel="shortcut icon" type="image/x-icon" href=' + pathToLibs + '/image/favicon.ico>'); -->
      
      // css
      document.write('<link rel="stylesheet" href=' + pathToLibs + '/lib/bootstrap.min.css>');
      document.write('<link rel="stylesheet" href=' + pathToLibs + '/lib/fontawesome/css/all.min.css>');
      document.write('<link rel="styleshe

In [37]:
from bs4 import BeautifulSoup
import requests
import sys

# Access page
cik = '0000320193'
type = '10-Q'
dateb = '20160101'

# Obtain HTML for search page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
edgar_resp = requests.get(base_url.format(cik, type, dateb))
edgar_str = edgar_resp.text

# Find the document link
doc_link = ''
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if '2015' in cells[3].text:
            doc_link = 'https://www.sec.gov' + cells[1].a['href']

print(doc_link)
# Exit if document link couldn't be found
if doc_link == '':
    print("Couldn't find the document link")
    sys.exit()


# Obtain HTML for document page
doc_resp = requests.get(doc_link)
doc_str = doc_resp.text

# Find the XBRL link
xbrl_link = ''
soup = BeautifulSoup(doc_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile', summary='Data Files')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if 'INS' in cells[3].text:
            xbrl_link = 'https://www.sec.gov' + cells[2].a['href']

print(xbrl_link)
# Obtain XBRL text from document
xbrl_resp = requests.get(xbrl_link)
xbrl_str = xbrl_resp.text

# Find and print stockholder's equity
soup = BeautifulSoup(xbrl_str, 'lxml')
tag_list = soup.find_all()
print(len(tag_list))
for tag in tag_list:
    if tag.name == 'us-gaap:stockholdersequity':
        print("Stockholder's equity: " + tag.text)
        

https://www.sec.gov/Archives/edgar/data/320193/000119312515023697/0001193125-15-023697-index.htm
https://www.sec.gov/Archives/edgar/data/320193/000119312515023697/aapl-20141227.xml
3121
Stockholder's equity: 123328000000
Stockholder's equity: 111547000000
